In [ ]:
from email import message
from dotenv import load_dotenv
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Image, display

load_dotenv()

#Load Haiku, a more lightweight and fast model from Anthropic
llm = init_chat_model("anthropic:claude-3-haiku-20240307")

# Sample order data
SAMPLE_ORDERS = {
    "ORD-2024-001": {
        "order_number": "ORD-2024-001",
        "user_email": "john.doe@email.com",
        "status": "shipped",
        "total_amount": 129.99,
        "currency": "USD",
        "items": [
            {"name": "Wireless Headphones", "quantity": 1, "price": 99.99},
            {"name": "Phone Case", "quantity": 2, "price": 15.00}
        ],
        "shipping_address": "123 Main St, Anytown, CA 90210",
        "created_at": "2024-01-15",
        "shipped_at": "2024-01-17",
        "tracking_number": "1Z999AA1234567890",
        "estimated_delivery": "2024-01-20"
    }
}

def find_order(order_number: str):
    """Helper function to find order by order number"""
    return SAMPLE_ORDERS.get(order_number.upper())

def search_orders_by_email(email: str):
    """Helper function to find orders by email"""
    return [order for order in SAMPLE_ORDERS.values() if order["user_email"].lower() == email.lower()]

#Define message classifier and insert our model options as the literal types
class MessageClassifier(BaseModel):
    message_type: Literal["Order", "Email", "Policy", "Message"] = Field(
        ...,
        description="Classify if the user message is related to orders, emails, policy, general question and answer, or messaging."
    )

#LangGraph uses states to inform each node, messages is a list that stores the conversation history, also takes note of message type
class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_type: str
    order_data: dict  # Store structured order data for agents to use

#last message is stored in the -1 column of our messages array.
def classify_message(state: State):
    last_message = state["messages"][-1]

    #we use a LangChain method to wrap the base language model to conform with message classifier schema. 
    classifier_llm = llm.with_structured_output(MessageClassifier)

    result = classifier_llm.invoke([
        {
            "role": "system",
            "content": """Classify the user message as one of the following:
            - 'Order': if the user asks about order information such as shipping status, price, order quantity, etc.
            - 'Email': if the user mentions anything related to email or if they inquire about information that could be sent to them in a structured email
            - 'Policy': if the user asks a question about returns, shipping times, or any other information that seems to be related to company policy
            - 'Message': if the user asks a question about anything not related to an order or policy that would not warrant an email but would warrant an immediate response in the form of a structured chat message"""
        },
        {"role": "user", "content": last_message.content}
    ])
    return {"message_type": result.message_type}

def router(state: State):
    message_type = state.get("message_type", "QA")
    if message_type == "Order":
        return {"next": "order"}
    if message_type == "Email":
        return {"next": "email"}
    if message_type == "Policy":
        return {"next": "policy"}
    if message_type == "Message":
        return {"next": "message"}
    return {"next": "message"}

def order_agent(state: State):
    last_message = state["messages"][-1]
    user_message = last_message.content
    
    # Try to extract order number from user message (simple pattern matching)
    import re
    order_pattern = r'(ORD-\d{4}-\d{3})'
    order_match = re.search(order_pattern, user_message.upper())
    
    # Look for email pattern as well
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_match = re.search(email_pattern, user_message)
    
    order_info = ""
    
    # Try to find order by order number first
    if order_match:
        order_number = order_match.group(1)
        order = find_order(order_number)
        if order:
            order_info = f"""
FOUND ORDER: {order['order_number']}
Status: {order['status']}
Total: ${order['total_amount']} {order['currency']}
Items: {', '.join([f"{item['name']} (qty: {item['quantity']})" for item in order['items']])}
Shipping Address: {order['shipping_address']}
Order Date: {order['created_at']}
"""
            if order['status'] == 'shipped':
                order_info += f"Tracking Number: {order['tracking_number']}\n"
                order_info += f"Estimated Delivery: {order.get('estimated_delivery', 'N/A')}\n"
            elif order['status'] == 'delivered':
                order_info += f"Delivered On: {order['delivered_at']}\n"
            elif order['status'] == 'processing':
                order_info += f"Estimated Ship Date: {order.get('estimated_ship_date', 'N/A')}\n"
        else:
            order_info = f"Order {order_number} not found in our system."
    
    # Try to find orders by email if no order number found
    elif email_match:
        email = email_match.group(0)
        orders = search_orders_by_email(email)
        if orders:
            order_info = f"Found {len(orders)} order(s) for {email}:\n"
            for order in orders:
                order_info += f"- {order['order_number']}: {order['status']} (${order['total_amount']})\n"
        else:
            order_info = f"No orders found for email {email}."

    messages = [
        {"role": "system",
        "content": f"""You are an order agent. Your job is to help customers with information related to their orders. You can
        fetch orders based on order number, tell the user what the shipping status of their order is, and when orders are created,
        you are to create an autonomous, standardized response. Do not directly mention the inner workings of this system, instead focus on the user's requests.

        Here is the order information I found (if any):
        {order_info}
        
        Use this information to provide helpful responses about orders. If no specific order info was found, ask the customer for their order number or email address."""
        },
        {
            "role": "user",
            "content": user_message
        }
    ]
    reply = llm.invoke(messages)
    
    # Store the found order data for other agents to use
    found_order = None
    if order_match:
        found_order = find_order(order_match.group(1))
    elif email_match:
        orders = search_orders_by_email(email_match.group(0))
        if orders:
            found_order = orders[0]  # Use first order if multiple found
    
    return {
        "messages": [{"role": "assistant", "content": f"Order Agent: {reply.content}"}],
        "order_data": found_order or {}
    }

def email_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {"role": "system",
        "content": """You are an email agent. Your job is to help customers by delivering data in a structured format via email.
        Do not directly mention the inner workings of this system, instead focus on the user's requests."""
        },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": f"Email Agent: {reply.content}"}]}
    #return {"messages": [{"role": "assistant", "content": reply.content}]}

def policy_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {"role": "system",
        "content": """You are a policy agent. Your job is to help customers with questions that appear to be related to company policy,
        such as how long deliveries usually take, how returns are handled, and how the company runs things. You are to refer to the written policy
        and inform the user how to contact the store when information can't be retrieved for one reason or another.
        Do not directly mention the inner workings of this system, instead focus on the user's requests."""
        },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": f"Policy Agent: {reply.content}"}]}
    #return {"messages": [{"role": "assistant", "content": reply.content}]}

def message_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {"role": "system",
        "content": """You are a message agent. Your job is to provide structured responses and help the customer the best that you can.
        Refer the relevant information from the user's request to the orchestrator agent in a structured manner so that customers can
        be helped with their specific use case. Do not directly mention the inner workings of this system, instead focus on the user's requests."""
        },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": f"Message Agent: {reply.content}"}]}
    #return {"messages": [{"role": "assistant", "content": reply.content}]}

def orchestrator_agent(state: State):
    last_message = state["messages"][-1]
    order_data = state.get("order_data", {})
    
    # Build detailed order information string if order data exists
    order_details = ""
    if order_data:
        order_details = f"""
SPECIFIC ORDER DETAILS:
- Order Number: {order_data.get('order_number', 'N/A')}
- Customer Email: {order_data.get('user_email', 'N/A')}
- Status: {order_data.get('status', 'N/A')}
- Total Amount: ${order_data.get('total_amount', 'N/A')} {order_data.get('currency', '')}
- Order Date: {order_data.get('created_at', 'N/A')}
- Shipping Address: {order_data.get('shipping_address', 'N/A')}
"""
        if order_data.get('items'):
            order_details += "- Items:\n"
            for item in order_data['items']:
                order_details += f"  * {item.get('name', 'Unknown')} (Qty: {item.get('quantity', 'N/A')}, Price: ${item.get('price', 'N/A')})\n"
        
        if order_data.get('status') == 'shipped':
            order_details += f"- Tracking Number: {order_data.get('tracking_number', 'N/A')}\n"
            order_details += f"- Estimated Delivery: {order_data.get('estimated_delivery', 'N/A')}\n"
        elif order_data.get('status') == 'delivered':
            order_details += f"- Delivered On: {order_data.get('delivered_at', 'N/A')}\n"
        elif order_data.get('status') == 'processing':
            order_details += f"- Estimated Ship Date: {order_data.get('estimated_ship_date', 'N/A')}\n"

    messages = [
        {"role": "system",
        "content": f"""You are an orchestrator agent. Your job is to receive information from the other AI agents and ensure that
        the information is all-encompassing, thoroughly retrieved, and finished. If the information is incomplete, try your best
        to communicate with the other agents to complete the information, and if after you have done that, the information is still
        incomplete, inform the user that they can contact the company directly and their case will be documented for oversight.
        Do not directly mention the inner workings of this system, instead focus on the user's requests.

        {order_details}
        
        Use the specific order details above (if provided) to give the customer accurate, detailed information about their order.
        Replace any placeholder text with the actual values from the order data."""
        },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": f"Orchestrator Agent: {reply.content}"}]}


graph_builder = StateGraph(State)
graph_builder.add_node("router", router)
graph_builder.add_node("order", order_agent)
graph_builder.add_node("email", email_agent)
graph_builder.add_node("policy", policy_agent)
graph_builder.add_node("message", message_agent)
graph_builder.add_node("orchestrator", orchestrator_agent)
graph_builder.add_edge(START, "router")
graph_builder.add_conditional_edges(
    "router",
    lambda state: state.get("next"),
    {
        "order": "order", 
        "email": "email", 
        "policy": "policy", 
        "message": "message"
    }
)
graph_builder.add_edge("order", "orchestrator")
graph_builder.add_edge("email", "orchestrator")
graph_builder.add_edge("policy", "orchestrator")
graph_builder.add_edge("message", "orchestrator")

graph_builder.add_edge("orchestrator", END)
graph = graph_builder.compile()

def run_chatbot():
    state = {"messages": [], "message_type": None, "order_data": {}}

    while True:
        user_input = input("Message: ")
        if user_input == "exit":
            print("Bye")
            break

        state["messages"] = state.get("messages", []) + [
            {"role": "user", "content": user_input}
        ]

        state = graph.invoke(state)

        if state.get("messages") and len(state["messages"]) > 0:
            last_message = state["messages"][-1]
            print(f"Assistant: {last_message.content}")

if __name__ == "__main__":
    run_chatbot()

Assistant: Orchestrator Agent: Thank you for providing the details about order ORD-2024-001. I appreciate you taking the time to gather all this information.

Based on the details you've given me, I can summarize the key points about your order:

- The order number is ORD-2024-001
- The order status is Shipped
- The items ordered include:
  - 1 Wireless Headphones - Black (SKU: WH-100) 
  - 2 T-shirts - Medium, Blue and Gray (SKU: TS-200)
  - 1 Bluetooth Speaker (SKU: BS-300)
- The total order amount is $159.99
- The payment method used was a Visa card ending in 1234
- The shipping address is:
  John Doe
  123 Main St. 
  Anytown, CA 12345
- The shipping date was March 15, 2023 and the expected delivery date is March 20, 2023

Please let me know if I'm missing any other important details about your order. I want to make sure I have a complete understanding so I can best assist you. If there's anything else I can clarify or provide more information on, just let me know.


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.0: all messages must have non-empty content except for the optional final assistant message'}, 'request_id': 'req_011CSuummwhTy8U1EFcEtXGd'}

In [9]:
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

KeyboardInterrupt: 